In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

%reload_ext autoreload
from utils import load_images
from features import extract_features, load_features, save_features

images_paths, images = load_images(True)
images_paths.head()

,image_name,label,path,train,label_id,image_id
0,image25107.jpg,apparel,images/apparel/image25107.jpg,True,9,0
1,image26799.jpg,apparel,images/apparel/image26799.jpg,True,9,1
2,image23088.jpg,apparel,images/apparel/image23088.jpg,True,9,2
3,image0421.jpg,apparel,images/apparel/image0421.jpg,True,9,3
4,image15994.jpg,apparel,images/apparel/image15994.jpg,True,9,4


In [2]:
def orb_descriptor(image):
    orb = cv2.ORB_create(nfeatures = 50, scoreType = cv2.ORB_HARRIS_SCORE, WTA_K = 4)
    img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    _, descs = orb.detectAndCompute(img, None)
    return descs

# Convertir las imágenes a escala de grises y aplicar SIFT en el subconjunto de imágenes
descriptors, index = extract_features(orb_descriptor, images, min_features=3, progress = True)
print("Tamaño de  descriptores:", descriptors.shape[1])
print("Número de  descriptores:", descriptors.shape[0])
print("Número Máximo de Descriptores Por Imágen :", pd.value_counts(index).max())
print("Número Medio  de Descriptores Por Imágen :", pd.value_counts(index).mean().round(0).astype(int))
print("Número Mínimo de Descriptores Por Imágen :", pd.value_counts(index).min())

save_features(descriptors, index, images_paths, "ORB")
#descriptors = np.concatenate(descriptors)


  0%|          | 0/550 [00:00<?, ?it/s]

Tamaño de  descriptores: 32
Número de  descriptores: 17102
Número Máximo de Descriptores Por Imágen : 35
Número Medio  de Descriptores Por Imágen : 31
Número Mínimo de Descriptores Por Imágen : 3


In [ ]:
from sklearn.neighbors import NearestNeighbors
from collections import Counter


# Crear un modelo de k-NN
knn_model = NearestNeighbors(n_neighbors=30)  # Buscar los 5 vecinos más cercanos
knn_model.fit(lista_desciptores_aplanado)  # Ajustar el modelo a los descriptores

# Función para encontrar los vecinos más cercanos para un descriptor dado
def find_nearest_neighbors(descriptor, k = 10):
    descriptor_flat = descriptor.flatten()
    # Encontrar los índices y distancias de los vecinos más cercanos
    distances, indices = knn_model.kneighbors([descriptor_flat], k)  # Obtener k vecinos más cercanos
    return distances, indices

# Ejemplo: Encontrar vecinos más cercanos para el primer descriptor en la primera imagen
contador = Counter()
lista = []
print(len(descriptor_busqueda[0]))
for descriptor in descriptor_busqueda[0]:
    distances, indices = find_nearest_neighbors(descriptor, 20)
    lista.extend([index[indice] for indice in indices[0]])

print(len(lista))
Counter(lista).most_common(10)
# Imprimir los índices de los vecinos más cercanos y sus distancias
#print("Índices de los vecinos más cercanos:", indices)
#print("Distancias a los vecinos más cercanos:", distances)


14
280


[(263, 5),
 (762, 4),
 (381, 4),
 (207, 3),
 (166, 3),
 (81, 3),
 (956, 3),
 (508, 3),
 (779, 3),
 (453, 2)]

In [11]:
orb_descriptor(images[1]).dtype, descriptors.dtype

(dtype('uint8'), dtype('uint8'))

In [15]:
descriptors[:10, :]

array([[ 18, 137,  46, 192, 177,  65, 185,  13, 192, 124,  28,  77,  19,
         37, 197, 164,  94, 139, 169,  98, 111, 228,  70, 159, 135, 158,
        245, 168,  96, 130, 249, 100],
       [ 62, 134,  62,  12,  57,  81, 106,   9, 128,  44,  30, 141, 177,
        249,  23, 180, 196, 182, 169,  45, 108, 108, 119,  28, 155, 155,
        195, 132,  97, 227,  73, 173],
       [ 96, 106, 144, 219, 152,  75, 237, 126,  33, 126, 115,  53, 238,
         82,  29, 140, 231,   2,  93, 128, 202,  37,  22, 168, 234, 195,
        181, 207, 149, 144, 151, 206],
       [ 96,  74,  88, 200, 144,  75,  61, 125,  34, 126,  95, 141, 239,
         18,  25, 132,  87,  17,  41,  35, 206,  36,  22, 153, 216, 194,
        149, 207, 100, 144, 151,  66],
       [106,  77,  95, 138, 128,  83, 170, 125,  18, 237, 158,  77, 224,
         34, 222, 135, 207,  59, 107, 110,  51,  36,  40,  40, 226, 170,
         49, 207, 144,  22, 102, 144],
       [ 34, 139,  94, 200, 128, 115, 185,  29,   0,  62,  28,  77, 211,
  

In [32]:
matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING)
feat = descriptors[0, :].reshape(1, -1)
mat = matcher.knnMatch(feat, descriptors, 50)


AttributeError: 'cv2.DMatch' object has no attribute '__dict__'

In [34]:
matcher.train()

< cv2.DMatch 000001DDB4624530>